In [236]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os 
import ast
import json
import time

https://stackoverflow.com/questions/62335537/scraping-text-from-kickstarter-projects-return-nothing

In [173]:
base = "/Users/ivankoh/Downloads/kickstarter data/"
filepaths = [base + f for f in os.listdir(base) if f.endswith('.csv')]
df = pd.concat(map(pd.read_csv, filepaths))
df_1 = df.drop_duplicates(subset=['id'])
df_1['main_url'] = df_1['urls'].apply(lambda x: eval(x)['web']['project'])
print(len(df))
print(len(df_1))

392244
203510


/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_74056/4110031066.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['main_url'] = df_1['urls'].apply(lambda x: eval(x)['web']['project'])


In [ ]:
def get_slug(url):
    return re.search('/projects/(.*)\?', url).group(1)

# get the slug from the urls. the slug is the input to the GraphQL API call 
slugs = list(map(get_slug, df_1['main_url'].tolist()))
# while iterating through all projects, add the story and risk to these lists. At the end len of both list 
# should be the same as the len of the dataframe. if all good then add as columns to the dataframe df. 
story_list = []
risk_list = []

In [269]:
block_size = len(df_1) // 100
num_blocks = len(df_1) // 2035
num_blocks
2 * block_size
print(0 * block_size, 1 * block_size)

0 2035


In [296]:
# instantiate a new Session 
s = requests.Session()
headers = {
            'content-type': 'application/json',
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
           'cookie': '_ksr_session=fc2U7qXXaRN91foNiE53NyU3s181NZO0Ll57xPkYxZ5iyUNgus35a0HwsPBTfViBY%2ByAKbtpRirAVLxOGKzG%2BYMOmsLRBPujZep%2Fca%2B1%2FXzW3xX56VXkh5w6ItYhIctEFifQQhw3rTmvoljyHw%3D%3D--4pK6xBEgChjqgmte--LH4Q1qSnhU%2FYX9JgTzGuSQ%3D%3D;'
          }
# GET request to kickstarter to get some token needed for later 
# r = s.get("https://www.kickstarter.com", headers=headers)
# BeautifulSoup parses the html returned as a response from the GET request 
# soup = BeautifulSoup(r.text, 'html.parser')
# xcsrf = soup.find("meta", {"name": "csrf-token"})["content"]

query = """
query Campaign($slug: String!) {
  project(slug: $slug) {
    risks
    story(assetWidth: 680)
  }
}"""

# PARAMS
counter = 0
block_number = 0

# pause for 5 seconds every 5 iterations
for i in range(275, (block_number + 1) * block_size):
    slug = slugs[i]
    if counter == 5:
        print("TAKING A NAP ZZZ...")
        time.sleep(60)
        counter = 0
    counter += 1
    print(f"--------{slug}------")
    # POST request to GraphQL API situated in kickstarter domain 
    # response from POST request should contain the story and risks
    r = s.post("https://www.kickstarter.com/graph",
        headers= {
              'content-type': 'application/json',
              'x-csrf-token': 'GgpaCc5tY-2VRVj1bE1aLOPKLkwRFEbIF205o2WorHLvaUEoOIPiZJSx4rbQg0csVbwBfq-jKGXPw2j42dvQCQ',
              'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36',
              'cookie': '_ksr_session=aVeEJvdlSYzJIE8X7f3v85C2T7b9Q30EN0ONWdeWjMLei6H06q6eRA3DZfCsbal40AfBTSbfczmlTv2ZIpvdyykn4QQYZdSlIzlalRivb6Xlb%2BXhXDhBNJg7KPYEEyFFIX3Qz8%2F61YNIoV136X5ju%2BE7jrqQFsGEB1%2FhAwJZ7kjD6QiwTDFdWWmJDr2wmREspLGXKUO0gicnsrUxQJPdTQMCcFNXy6sx95x5S%2FiNs4xVq0t4jWv%2BaiSnwBp8pC6Xu81FBhpD%2F9sNLZ9YyS5Obzbv0jI%3D--tM1UdSPo8XKxtj%2Bt--WQhnQX4EovqKwzBU9tiWIg%3D%3D'
        },
        json = {
            "operationName":"Campaign",
            "variables":{
                "slug": slug
            },
            "query": query
        })
    try:
        result = r.json()
        print("-------STORY--------")
        story_html = result["data"]["project"]["story"]
        soup = BeautifulSoup(story_html, 'html.parser')
        curr_story = []
        for i in soup.find_all('p'):
            print(i.get_text())
            curr_story.append(i.get_text())
        story_list.append(" ".join(curr_story) if curr_story else "")

        print("-------RISKS--------")
        print(result["data"]["project"]["risks"])
        risk_list.append(result["data"]["project"]["risks"] if result["data"]["project"]["risks"] else "")
    except:
        print(r)
        story_list.append("")
        risk_list.append("")
        continue

--------littlelunarrabbit/guardian-constellations-a-magical-girl-inspired-pi------
-------STORY--------
Thank you for taking the time to stop and look at my campaign! I'm Sam, sole artist and owner of Little Lunar Rabbit Designs.
Ultimately, the goal of this campaign is to get some pins made. To help make the project more manageable, the initial goal was a relatively low one - being only one pin. Rather than running the campaign for all 10 designs and risk not being able to fund everything, I had hoped that by breaking the project into smaller chunks, we would be able to produce as many as we could!
We're now a ways into the campaign, and the original campaign is beyond successfully funded! Since we do still have some time left, I have added 4 all new bonus designs, re-released the original Crystal Bunny pin, and added on some really fun milestone stretch goals! 
The pin that started it all - my crystal bunny design! Having made this one, having gone through the whole process of making

-------STORY--------
Mes expositions  « Collages & Photomontages » en Islande 2019
Des collages ! Voilà plus de 40 ans que j'en réalise. Tout d'abord sur papier, dans mes jeunes années, cela aboutira à 3 expositions en Provence (France) dans les années 80' qui remporteront un certain succès. Je pressens les potentialités du numérique dans ce domaine particulier, mais je reste « timide » bien que techniquement toujours très investi...
Puis arrive la Révolution Numérique généralisée, mes Collages & Photomontages reprennent depuis 2017 mais sous une forme résolument numérique et désormais en ultra haute définition avec des programmes informatiques extraordinaires de finesse et de précision.
En 2018, j'éprouve l'impérieux désir de franchir le pas : exposer, montrer, s'exposer aussi d'une certaine façon ... au regard d'autrui. Un plaisir et une souffrance...
Le lieu sera vite choisi, l'ISLANDE, mon pays d'adoption où j'ai déjà vécu plusieurs années et où une bonne partie de ma famille résid

-------STORY--------
David Mack and Riley Reid have been friends and collaborators of mine for years. The beauty of a relationship predicated on creative collaboration is that every interaction can turn into something bigger.  
David and I constantly bounce ideas off one another, and this initiative is no different. The three of us sat in a small room covered in cats, and wondered whether we could collaborate on something both compelling and hyper focused, playing to each of our strengths.
Photography is a curious art in that it is infinitely repeatable, especially in the digital age. I've always been interested in ways to circumvent that.  My first book, Temple of Art, showcased art painted onto portraits of the artists by the artists themselves. This project explores that idea via the medium of a single subject; a photographic canvas with Riley as the ineffable foundation for David's original art.
David will create 100 unique ink paintings on top of an 11x14 archival photograph of Ri

-------STORY--------
My Own Private Hollywood (or MOPH, for short) is an independently owned film festival that travels from city to city exhibiting short and feature-length films and movies from creatives all over the world--although, for now, we're limiting submissions to those from the U.S.  Each year, we select projects that are fresh, illuminating and unconventional enough to not only showcase the producers' talents but also tell original stories and captivate new audiences.  Our objective is to not necessarily push creatives to Hollywood but pull "Hollywood" to them by allowing individuals to start their own production companies and improve their local economies.  
Our mission is to enhance the marketing, distributing and merchandising opportunities for creatives worldwide, allowing them to successfully enter the entertainment industry without facing adversity, prejudice or nepotism.  We feel that if "unknowns" display their works in venues and arenas outside the internet, they c

--------1109932044/first-solo-photography-exhibition------
-------STORY--------
I've been offered the opportunity of my first solo exhibition for my fine art photography.  It will take place at Shaw's Arts Cafe in Blackpool from 2 March 2018 for six weeks.
I shoot exclusively on black and white film and print using processes from the Victorian era such as cyanotype, salt print, van dyke brown and kallitype.
As a final year Photography BA(Hons) degree student at Blackpool and the Fylde College, a solo exhibition will contribute enormously to both my university work and potentially begin my career as a creative practitioner.  In order to turn this opportunity into a reality I need to raise the money to:
On display will be highlights of my various experimental photography from the past twelve months.
If you're able to support this opportunity I will be extremely grateful.
-------RISKS--------
Time will be the biggest issue.  I've set aside several whole days each week to create the prints

...

--------1724334663/the-plant-base-simple-honest-goodness------
-------STORY--------
Hi all!
We're looking for a little help to get us over the finish line. 
If you're a health-conscious hero, a plant-based person, a youthful yogi or a muscle-clad maestro you'll love The Plant Base!
We will be striving to help the nation stay on track with their health goals, one delicious bowl at a time.
Secondly, we will be putting as much effort as we can into being a sustainable business by making sure that our fresh, whole ingredients are as locally sourced possible and that all our packaging is plant-based and compostable.
We plan to be a hub for the community for years to come, so what better way to start than harnessing people power!
-------RISKS--------
Running a restaurant means long hours and hard work, the challenges will be numerous.

I have run a successful, busy kitchen before and have been self-employed for practically my whole working life. I am confident and more determined than ever t

-------STORY--------
Update 5-12-14:
Wow! Two weeks left, and almost half of funds raised. Most of the projects that fail on Kickstarter never raise more than about 10% of their goal, so we are on a great path. I should mention here that our family and friends have been more generous than I could imagine, and your faith in us is a massive inspiration! To celebrate you guys, we have created a special Thank You image, tagging all of our supporters. Head over to the Logan & Royce Facebook page to check it out! 
Update 5-5-14:
Almost at $2000! Special thanks to the band Bon Saints, who have contacted us and agreed to allow use of their music free of charge, to nerd-tastic fan haven dorkly.com for helping spread the word on Twitter, and to production company Atomic Imaging for getting involved and offering their industry standard facilities and equipment for a fraction of the price! 
This has been a lot of work so far, but even more fun. This week we will be hitting blogs, personalities, an

--------1110034816/had-i-been-raised-in-the-village-la-exhibition------
-------STORY--------
Your support is crucial to the success of this campaign! Please take a closer look at the project description below as well as the terrific rewards on the right that I am offering in return for your generous contribution. Thank you!
Project Description:
I’ve been invited to have my first one-woman show with Reginald Ingraham gallery in Los Angeles May 25th. I am really excited about this great opportunity to share my work on the West Coast.The exhibition is titled, "Had I Been Raised In the Village..." and will showcase many of the paintings in my current body of work. 
As a Rwandan living in Harlem, New York, I work to configure ways of demonstrating my in-between state of being stuck in a perpetual transit lounge. I mine diverse sources such as imigongo cow dung painting patterns, Matisse's color palette, and the visual effects on the severely deteriorated exterior walls of old colonial build

-------STORY--------
-------RISKS--------
We guarantee the quality of each belt we make and we are confident in meeting the deadlines if this fundraising is successful, as all our craftsmen are experienced in both techniques and time management. We will produce and deliver the belts strictly according to our timeline. The success of the campaign will enable us to fulfil all rewards and expand our business by introducing more products.
We will keep you updated throughout the duration of our campaign.
--------1943003045/chris-cole-ep------
-------STORY--------
Through much encouragement from friends and family, I'm finally recording some of the songs that I've been writing and preforming over the last few years.  It is a 5 song EP of soulful country and folk songs and are the result of my own travels and experiences.  They are simple but thoughtful songs about life, love, and moving from one place to the next. I hope they provide a relaxing and fun experience that adds to the soundtrack 

KeyboardInterrupt: 

## Latest Iteration Ran Till: 1074, Local copy latest: 0_1074

In [294]:
#SAVE LIST IN LOCAL FOR USE LATER. NAME FILE AS {from}_{to]_{story/risk}.json
story_file_name = "0_1074_story" #CHANGE THIS
risk_file_name = "0_274_risk" #CHANGE THIS 

story_file_name += ".json"
risk_file_name += ".json"

with open(story_file_name, 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(story_list, f, indent=2) 

with open(risk_file_name, 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(risk_list, f, indent=2) 
    
# with open(story_file_name, 'r') as f:
#     story_subset = json.load(f)

In [297]:
print("risk:", len(risk_list),"\n", "story:", len(story_list))

risk: 1075 
 story: 1075


In [293]:
print("LATEST CALL DF ROW", "\n")
print(df_1.iloc[274]['blurb'])

print("LATEST STORY IN LIST", "\n")
print(story_list[-1])

print("LATEST RISK IN LIST", "\n")
print(risk_list[-1])

LATEST CALL DF ROW 

A Hard Enamel Keychain of Mr. Drippy from the PS3 game Ni No Kuni
LATEST STORY IN LIST 

Hello, My name is Daryl and I run a small Etsy store where I sell enamel pins and other products of my designs. Typically, I try to focus on designing and selling original characters, but I recently played through the PS3 classic Ni No Kuni and was inspired to design my first keychain.  Here are some specifics: Assurance: All pledge money received will go straight into the production and manufacturing of this keychain. The manufacturer that I work with does require 100 piece minimum, so I don't foresee having much money leftover. If there does happen to be some funds left over, all the remainder will go toward purchasing new backing cards and other such items for packaging.
LATEST RISK IN LIST 

I've been designing and manufacturing hard enamel pins for almost a year now and have yet to encounter any sort of challenge that wasn't easily overcome. However, this being my first ke

In [165]:
s = requests.Session()

In [234]:
import requests
import json

url = "https://www.kickstarter.com/graph"

payload = json.dumps([
  {
    "operationName": "Campaign",
    "variables": {
      "slug": "leightonconnor/akashic-titan-blue-bolt"
    },
    "query": "query Campaign($slug: String!) {\n  project(slug: $slug) {\n    id\n    isSharingProjectBudget\n    risks\n    story(assetWidth: 680)\n    currency\n    spreadsheet {\n      displayMode\n      public\n      url\n      data {\n        name\n        value\n        phase\n        rowNum\n        __typename\n      }\n      dataLastUpdatedAt\n      __typename\n    }\n    environmentalCommitments {\n      id\n      commitmentCategory\n      description\n      __typename\n    }\n    __typename\n  }\n}\n"
  }
])
headers = {
  'content-type': 'application/json',
  'x-csrf-token': 'bL5nMRm5s147N8RxvNcth-E8NAOt3SprGYtvvKsVLnuZ3XwQ71cy1zrDfjIAGTCHV0obMRNqRMbBJT7nF2ZSAA',
  'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
  'cookie': '_ksr_session=eus10UNK7iB%2FwIMXafXg5yJdYvRD7rHlR%2Ba1KEbUnB5EKTgQxNKEIqrp%2BIyjRC%2FNXw76u4Eki4AsiBom5M1LKNJKWCD65CBzeegWrk2%2FRV9Sht6dlOKplCATNdrmEIIa0Bt%2FKKGnT2m9ID1nTCduj73%2BdeJK663MNcbYWXBHkFyZJgUKh4JcCx3hL0UF2quVeBvfV45M0CNrT1j4DchvTlFBn%2BpE37ZaGldNz7BF0RAcCjIZrt%2FzNkxugCaFfWEPjxh9amOCvytIcdjxOaE68AAmHuU%3D--RPNmoWC%2FGnWTmsfw--9466UG%2Bqs2V%2F8B8zFtXUig%3D%3D'
}

response = requests.request("POST", url, headers=headers, data=payload)
print(response.status_code)
print(response.json()[0]['data']['project']['risks'])

403


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [256]:
import requests
import json

url = "https://www.kickstarter.com/graph"

payload = json.dumps([
  {
    "operationName": "Campaign",
    "variables": {
      "slug": "leightonconnor/akashic-titan-blue-bolt"
    },
    "query": "query Campaign($slug: String!) {\n  project(slug: $slug) {\n    id\n    isSharingProjectBudget\n    risks\n    story(assetWidth: 680)\n    currency\n    spreadsheet {\n      displayMode\n      public\n      url\n      data {\n        name\n        value\n        phase\n        rowNum\n        __typename\n      }\n      dataLastUpdatedAt\n      __typename\n    }\n    environmentalCommitments {\n      id\n      commitmentCategory\n      description\n      __typename\n    }\n    __typename\n  }\n}\n"
  }
])
headers = {
  'content-type': 'application/json',
  'x-csrf-token': 'GgpaCc5tY-2VRVj1bE1aLOPKLkwRFEbIF205o2WorHLvaUEoOIPiZJSx4rbQg0csVbwBfq-jKGXPw2j42dvQCQ',
  'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36',
  'cookie': '_ksr_session=aVeEJvdlSYzJIE8X7f3v85C2T7b9Q30EN0ONWdeWjMLei6H06q6eRA3DZfCsbal40AfBTSbfczmlTv2ZIpvdyykn4QQYZdSlIzlalRivb6Xlb%2BXhXDhBNJg7KPYEEyFFIX3Qz8%2F61YNIoV136X5ju%2BE7jrqQFsGEB1%2FhAwJZ7kjD6QiwTDFdWWmJDr2wmREspLGXKUO0gicnsrUxQJPdTQMCcFNXy6sx95x5S%2FiNs4xVq0t4jWv%2BaiSnwBp8pC6Xu81FBhpD%2F9sNLZ9YyS5Obzbv0jI%3D--tM1UdSPo8XKxtj%2Bt--WQhnQX4EovqKwzBU9tiWIg%3D%3D'
}

response = requests.request("POST", url, headers=headers, data=payload)
print(response.status_code)
print(response.json()[0]['data']['project']['risks'])

200
This is my fourth solo Kickstarter campaign, and I delivered all three of my previous games in a timely manner. 

I have a complete draft of the text, with the exception of Appendix 1. I have a team of editors lined up who will give me feedback so that I can revise, expand, and polish it as needed. The most time-consuming tasks will be completing the artwork and the layout. I've done this before, with the previous Akashic Titan zine and with my game Project 8Ball. Depending on the stretch goals I will be depending on someone else for illustrations and additional text, but I have worked with Josh before and I am confident he will hit his deadlines. 

I believe I will be able to finish the book by June, but just to be cautious I have set the estimated delivery month as July. 

Barring any unforeseen catastrophes, I should have no problem printing and shipping the zine by July.


In [255]:
s = requests.Session()
headers = {
            'content-type': 'application/json',
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
          }
# GET request to kickstarter to get some token needed for later 
r = s.get("https://www.kickstarter.com", headers=headers)
#BeautifulSoup parses the html returned as a response from the GET request 
soup = BeautifulSoup(r.text, 'html.parser')
xcsrf = soup.find("meta", {"name": "csrf-token"})["content"]

In [ ]:
'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'
'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'

In [237]:
print("hello")
time.sleep(3)
print("goodnight")

hello
goodnight
